First, create a new environment and install:
> `pip install mearec`


Sidenote, if you get an error while creating templates.. you may need to compile some stuff by going `~/.config/mearec/1.9.0/cell_models/bbp/mods` and running `nrnivmodl`. See note [here](https://github.com/SpikeInterface/MEArec/issues/166#issuecomment-1901578766).

In [ ]:
import MEArec as mr

## Generate waveform templates

In [ ]:

templates_params = mr.get_default_templates_params()
cell_models_folder = mr.get_default_cell_models_folder()

In [ ]:
# takes a few minutes

templates_params['probe'] = 'Neuropixels-384'
templates_params['n'] = 300

tempgen = mr.gen_templates(cell_models_folder=cell_models_folder, params=templates_params, verbose=True, n_jobs=1)

In [ ]:
# save templates
templates_filename = 'data/templates_300_neuropixels.h5'
mr.save_template_generator(tempgen=tempgen, filename=templates_filename)

## Generate timeseries recordings

In [ ]:
# load templates
tempgen = mr.load_templates(templates_filename)

In [ ]:
recording_params = mr.get_default_recordings_params()
# pprint(recording_params)

In [ ]:
# Set parameters
recording_params['spiketrains']['n_exc'] = 200
recording_params['spiketrains']['n_inh'] = 50
recording_params['spiketrains']['seed'] = 0

recording_params['templates']['min_amp'] = 40
recording_params['templates']['max_amp'] = 300
recording_params['templates']['seed'] = 0

recording_params['recordings']['modulation'] = 'electrode'
recording_params['recordings']['noise_mode'] = 'distance-correlated' #'uncorrelated'
recording_params['recordings']['noise_half_distance'] = 500
recording_params['recordings']['fs'] = 25000
recording_params['recordings']['bursting'] = True
recording_params['recordings']['n_bursting'] = 20
# recording_params['recordings']['f_inf'] = 25
# recording_params['recordings']['st_inf'] = 5
recording_params['recordings']['noise_color'] = True
recording_params['recordings']['color_peak'] = 60
recording_params['recordings']['noise_level'] = 15
recording_params['recordings']['filter_cutoff'] = [300, 6000]

# use chunk options
recording_params['recordings']['chunk_conv_duration'] = 10
recording_params['recordings']['chunk_noise_duration'] = 10
recording_params['recordings']['chunk_filter_duration'] = 10
recording_params['recordings']['seed'] = 0

In [ ]:
%%time

durations = [10, 100, 200]

for dur in durations:
    recording_params['spiketrains']['duration'] = dur
    recgen = mr.gen_recordings(templates=templates_filename, params=recording_params, verbose=True)
    print('Recording shape', recgen.recordings.shape)
    mr.save_recording_generator(recgen, f'data/ephys_sim_neuropixels_{dur}s_384ch.h5')